In [0]:
from google.colab import drive 
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [0]:
train = pd.read_csv("gdrive/My Drive/Colab Notebooks/Zindi/Mobile Money and Financial Inclusion in Tanzania/training.csv")
test = pd.read_csv("gdrive/My Drive/Colab Notebooks/Zindi/Mobile Money and Financial Inclusion in Tanzania/test.csv")
train.head()

,ID,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q8_10,Q8_11,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Latitude,Longitude,mobile_money,savings,borrowing,insurance,mobile_money_classification
0,5086,98,2,3,1,1,2,2,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,2,-1,2,-1,1,-1,4,4,-4.460442,29.811396,0,0,0,0,0
1,1258,40,1,1,3,5,1,1,1,0,0,0,0,0,0,0,0,0,0,1,-1,-1,1,4,1,5,4,4,1,4,-6.176438,39.244871,1,1,1,0,3
2,331,18,2,4,6,3,2,1,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,2,-1,2,-1,1,-1,1,1,-6.825702,37.652798,1,0,0,0,2
3,6729,50,1,1,3,1,1,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,1,2,2,-1,4,-1,1,4,-3.372049,35.808307,1,0,1,0,3
4,8671,34,1,1,1,1,2,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,2,-1,1,1,1,-1,1,4,-7.179645,31.039095,1,1,0,1,3


In [0]:
train_ID = train['ID']
test_ID = test['ID']
# Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train.drop(['ID'], axis=1, inplace=True)
test.drop(['ID'], axis=1, inplace=True)

In [0]:
train.shape

(7094, 36)

In [0]:
test_ID.shape

(2365,)

In [0]:
from sklearn.preprocessing import LabelEncoder
le_no_financial_services  = LabelEncoder()
le_other_only = LabelEncoder()
le_mm_only  = LabelEncoder()
le_mm_plus  = LabelEncoder()

In [0]:
train['no_financial_services'] = le_no_financial_services.fit_transform(train['mobile_money_classification'] ==0)
train['other_only'] = le_other_only.fit_transform(train['mobile_money_classification'] ==1)
train['mm_only'] = le_mm_only.fit_transform(train['mobile_money_classification'] ==2)
train['mm_plus'] = le_mm_plus.fit_transform(train['mobile_money_classification'] ==3)

In [0]:
train.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q8_10,Q8_11,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Latitude,Longitude,mobile_money,savings,borrowing,insurance,mobile_money_classification,no_financial_services,other_only,mm_only,mm_plus
0,98,2,3,1,1,2,2,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,2,-1,2,-1,1,-1,4,4,-4.460442,29.811396,0,0,0,0,0,1,0,0,0
1,40,1,1,3,5,1,1,1,0,0,0,0,0,0,0,0,0,0,1,-1,-1,1,4,1,5,4,4,1,4,-6.176438,39.244871,1,1,1,0,3,0,0,0,1
2,18,2,4,6,3,2,1,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,2,-1,2,-1,1,-1,1,1,-6.825702,37.652798,1,0,0,0,2,0,0,1,0
3,50,1,1,3,1,1,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,1,2,2,-1,4,-1,1,4,-3.372049,35.808307,1,0,1,0,3,0,0,0,1
4,34,1,1,1,1,2,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,2,-1,1,1,1,-1,1,4,-7.179645,31.039095,1,1,0,1,3,0,0,0,1


In [0]:
y_train = train.iloc[:,-4:]
train = train.drop(['mobile_money_classification', 'mobile_money', 'savings', 'borrowing', 'insurance', 'no_financial_services',	'other_only',	'mm_only',	'mm_plus'], axis =1)
X_train = train
X_test = test

In [0]:
X_train.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q8_10,Q8_11,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Latitude,Longitude
0,98,2,3,1,1,2,2,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,2,-1,2,-1,1,-1,4,4,-4.460442,29.811396
1,40,1,1,3,5,1,1,1,0,0,0,0,0,0,0,0,0,0,1,-1,-1,1,4,1,5,4,4,1,4,-6.176438,39.244871
2,18,2,4,6,3,2,1,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,2,-1,2,-1,1,-1,1,1,-6.825702,37.652798
3,50,1,1,3,1,1,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,1,2,2,-1,4,-1,1,4,-3.372049,35.808307
4,34,1,1,1,1,2,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,2,-1,1,1,1,-1,1,4,-7.179645,31.039095


In [0]:

X_test.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q8_10,Q8_11,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Latitude,Longitude
0,30,2,1,3,6,2,1,0,0,1,0,0,0,0,0,0,0,0,-1,-1,1,1,6,1,4,5,-1,1,4,-2.596421,32.913584
1,53,1,1,3,1,2,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,2,-1,1,4,1,-1,1,4,-1.778303,34.062640
2,23,2,1,3,5,1,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,1,4,1,4,1,-1,1,1,-1.851789,31.536866
3,30,2,3,3,4,1,2,0,1,0,0,0,0,0,0,1,0,0,-1,1,-1,2,-1,2,-1,1,-1,1,4,-1.235656,34.229012
4,28,2,1,5,4,2,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,1,4,1,6,4,3,1,2,-7.780934,35.691878


**Spliting the data back to train(X,y) and test(X_sub)**

In [0]:
# features = pd.concat([X_test, X_train],sort=False axis=1).reset_index(drop=True)
# features.shape

In [0]:
# features.to_csv("gdrive/My Drive/Colab Notebooks/Zindi/Mobile Money and Financial Inclusion in Tanzania/new_features2.csv", index=False)
# features.head()

In [0]:
E_features1 = pd.read_csv("gdrive/My Drive/Colab Notebooks/Zindi/Mobile Money and Financial Inclusion in Tanzania/Enriched_trainingx3.csv")
E_features2 = pd.read_csv("gdrive/My Drive/Colab Notebooks/Zindi/Mobile Money and Financial Inclusion in Tanzania/Enriched_testx3.csv")
print(E_features1.shape,'---',E_features2.shape)

(7094, 41) --- (2365, 41)


In [0]:
E_features1 = pd.get_dummies(E_features1, columns=["sourceCountry", "areaType", "bufferUnits", "bufferUnitsAlias", "aggregationMethod"], sparse=True)
E_features2 = pd.get_dummies(E_features2, columns=["sourceCountry", "areaType", "bufferUnits", "bufferUnitsAlias", "aggregationMethod"], sparse=True)

In [0]:
E_features1.iloc[:,-10:].head()

,bufferRadii,Population to polygon size rating for the country,Apportionment confidence for the country,HasData,2016 Purchasing Power: Total,sourceCountry_TZ,areaType_RingBuffer,bufferUnits_esriKilometers,bufferUnitsAlias_Kilometers,aggregationMethod_BlockApportionment:TZ.Regions
0,1,2.834,2.614,1,1.559503e+09,1,1,1,1,1
1,1,2.834,2.614,1,6.687531e+10,1,1,1,1,1
2,1,2.834,2.614,1,5.687314e+10,1,1,1,1,1
3,1,2.834,2.614,1,1.392611e+09,1,1,1,1,1
4,1,2.834,2.614,0,0.000000e+00,1,1,1,1,1


In [0]:
X_train = pd.concat([X_train, E_features1.iloc[:,-10:]],sort=False, axis=1).reset_index(drop=True)
X_test = pd.concat([X_test, E_features2.iloc[:,-10:]],sort=False, axis=1).reset_index(drop=True)

In [0]:
# # Spliting the data back to train(X,y) and test(X_sub)
# X_train = E_features1.iloc[:len(y_train), :]
# X_test = E_features2.iloc[:len(test_ID), :]
X_train['2016 Purchasing Power: Total'] = np.log1p(X_train['2016 Purchasing Power: Total'])/10
X_test['2016 Purchasing Power: Total'] = np.log1p(X_test['2016 Purchasing Power: Total'])/10

X_train['Longitude'] = X_train['Longitude']/10
X_test['Longitude'] = X_test['Longitude']/10

print('Features size for train(X,y) and test(X_test):')
print('X', X_train.shape, 'y', y_train.shape, 'X_test', X_test.shape)

Features size for train(X,y) and test(X_test):
X (7094, 41) y (7094, 4) X_test (2365, 41)


In [0]:
X_train.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q8_10,Q8_11,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Latitude,Longitude,bufferRadii,Population to polygon size rating for the country,Apportionment confidence for the country,HasData,2016 Purchasing Power: Total,sourceCountry_TZ,areaType_RingBuffer,bufferUnits_esriKilometers,bufferUnitsAlias_Kilometers,aggregationMethod_BlockApportionment:TZ.Regions
0,98,2,3,1,1,2,2,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,2,-1,2,-1,1,-1,4,4,-4.460442,2.981140,1,2.834,2.614,1,2.116763,1,1,1,1,1
1,40,1,1,3,5,1,1,1,0,0,0,0,0,0,0,0,0,0,1,-1,-1,1,4,1,5,4,4,1,4,-6.176438,3.924487,1,2.834,2.614,1,2.492610,1,1,1,1,1
2,18,2,4,6,3,2,1,0,0,0,0,0,0,0,0,1,0,0,-1,-1,-1,2,-1,2,-1,1,-1,1,1,-6.825702,3.765280,1,2.834,2.614,1,2.476409,1,1,1,1,1
3,50,1,1,3,1,1,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,1,2,2,-1,4,-1,1,4,-3.372049,3.580831,1,2.834,2.614,1,2.105445,1,1,1,1,1
4,34,1,1,1,1,2,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,2,-1,1,1,1,-1,1,4,-7.179645,3.103910,1,2.834,2.614,0,0.000000,1,1,1,1,1


In [0]:
X_test.head()

,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8_1,Q8_2,Q8_3,Q8_4,Q8_5,Q8_6,Q8_7,Q8_8,Q8_9,Q8_10,Q8_11,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18,Q19,Latitude,Longitude,bufferRadii,Population to polygon size rating for the country,Apportionment confidence for the country,HasData,2016 Purchasing Power: Total,sourceCountry_TZ,areaType_RingBuffer,bufferUnits_esriKilometers,bufferUnitsAlias_Kilometers,aggregationMethod_BlockApportionment:TZ.Regions
0,30,2,1,3,6,2,1,0,0,1,0,0,0,0,0,0,0,0,-1,-1,1,1,6,1,4,5,-1,1,4,-2.596421,3.291358,1,2.834,2.614,1,2.372197,1,1,1,1,1
1,53,1,1,3,1,2,1,0,1,0,0,0,0,0,0,0,0,0,-1,1,-1,2,-1,1,4,1,-1,1,4,-1.778303,3.406264,1,2.834,2.614,1,2.032950,1,1,1,1,1
2,23,2,1,3,5,1,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,1,4,1,4,1,-1,1,1,-1.851789,3.153687,1,2.834,2.614,1,1.965481,1,1,1,1,1
3,30,2,3,3,4,1,2,0,1,0,0,0,0,0,0,1,0,0,-1,1,-1,2,-1,2,-1,1,-1,1,4,-1.235656,3.422901,1,2.834,2.614,1,1.877135,1,1,1,1,1
4,28,2,1,5,4,2,1,0,0,0,1,0,0,0,0,0,0,0,-1,-1,-1,1,4,1,6,4,3,1,2,-7.780934,3.569188,1,2.834,2.614,1,2.416453,1,1,1,1,1


In [0]:
y_train.head()
target_names = list(y_train)
target_names

['no_financial_services', 'other_only', 'mm_only', 'mm_plus']

In [0]:

y_train = np.array(y_train)


In [0]:
y_train

array([[1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       ...,
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [1, 0, 0, 0]])

In [0]:

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegressionCV

model = OneVsRestClassifier(LogisticRegressionCV(Cs=30,solver = 'saga', multi_class = 'multinomial', cv=10), n_jobs =-1)

In [0]:
model.fit(X_train.values, y_train)

OneVsRestClassifier(estimator=LogisticRegressionCV(Cs=30, class_weight=None,
                                                   cv=10, dual=False,
                                                   fit_intercept=True,
                                                   intercept_scaling=1.0,
                                                   l1_ratios=None, max_iter=100,
                                                   multi_class='multinomial',
                                                   n_jobs=None, penalty='l2',
                                                   random_state=None,
                                                   refit=True, scoring=None,
                                                   solver='saga', tol=0.0001,
                                                   verbose=0),
                    n_jobs=-1)

In [0]:
y_predicted = model.predict_proba(X_test.values)

In [0]:
y_predicted.shape

(2365, 4)

**Submission**

In [0]:
submission = pd.DataFrame({'ID' : np.array(test_ID),'no_financial_services' : y_predicted[:,0], 'other_only' : y_predicted[:,1], 'mm_only' : y_predicted[:,2], 'mm_plus' : y_predicted[:,3]})

In [0]:
submission.head()

,ID,no_financial_services,other_only,mm_only,mm_plus
0,2352,0.000016,0.000148,0.385341,0.956307
1,8208,0.027388,0.110875,0.144676,0.617977
2,2785,0.000704,0.012922,0.130666,0.932063
3,2967,0.349973,0.543546,0.040150,0.108749
4,1697,0.000047,0.000194,0.225343,0.975079


In [0]:
submission.to_csv("gdrive/My Drive/Colab Notebooks/Zindi/Mobile Money and Financial Inclusion in Tanzania/new_submission.csv", index=False)